In [1]:
import numpy as np

In [2]:
state = {
    0:"Burger",
    1:"Pizza",
    2:"Hot Dog",
}
state

{0: 'Burger', 1: 'Pizza', 2: 'Hot Dog'}

### Transition Matrix

In [3]:
A = np.array([
    [0.2, 0.6, 0.2],
    [0.3, 0.0, 0.7],
    [0.5, 0.0, 0.5]
])
A

array([[0.2, 0.6, 0.2],
       [0.3, 0. , 0.7],
       [0.5, 0. , 0.5]])

## Random Walk on Markov Chain
Start from some arbitrary state and move along the markov chain following the transition probabilities

In [4]:
n_steps = 20
start_state = 0 # Burger
print(state[start_state],'--->',end=' ')
prev_state = start_state

while n_steps-1: # n_steps - 1 times
    curr_state = np.random.choice([0,1,2], p = A[prev_state])
    print(state[curr_state],'--->',end=' ')
    prev_state = curr_state
    n_steps-=1

Burger ---> Pizza ---> Hot Dog ---> Burger ---> Hot Dog ---> Burger ---> Pizza ---> Hot Dog ---> Hot Dog ---> Hot Dog ---> Hot Dog ---> Burger ---> Hot Dog ---> Hot Dog ---> Hot Dog ---> Burger ---> Burger ---> Pizza ---> Burger ---> Pizza ---> 

# Stationary Distribution

## Approach 1: Monte Carlo method

In [5]:
n_steps = 10**6 # 1 million steps
start_state = 0
prev_state = start_state
pi = np.array([0,0,0])
pi[start_state] = 1

i=0
while i<n_steps: # n_steps - 1 times
    curr_state = np.random.choice([0,1,2], p = A[prev_state])
    pi[curr_state]+=1
    prev_state = curr_state
    i+=1

In [6]:
print(f'Stationary Distribution = {pi/n_steps}')

Stationary Distribution = [0.352325 0.211705 0.435971]


## Approach 2: Repeated Matrix Multiplication
Converges faster than Monte carlo method

In [7]:
steps = 1000
A_power = np.matmul(A,A)
for i in range(steps-1):
    A_power = np.matmul(A_power, A)

In [8]:
print(f'A^{steps} \n={A_power}')
print(f'\nStationary Distribution = {A_power[0]}')

A^1000 
=[[0.35211268 0.21126761 0.43661972]
 [0.35211268 0.21126761 0.43661972]
 [0.35211268 0.21126761 0.43661972]]

Stationary Distribution = [0.35211268 0.21126761 0.43661972]


## Approach 3: Finding Left Eigenvectors

In [9]:
from scipy.linalg import eig

In [10]:
values, vectors = eig(A, left = True, right = False)
print(f'Eigenvalues = {values}')
print(f'Eigenvectors \n = {vectors}')

Eigenvalues = [ 1.  +0.j        -0.15+0.3122499j -0.15-0.3122499j]
Eigenvectors 
 = [[-0.58746336+0.j         -0.16984156-0.35355339j -0.16984156+0.35355339j]
 [-0.35247801+0.j          0.67936622+0.j          0.67936622-0.j        ]
 [-0.72845456+0.j         -0.50952467+0.35355339j -0.50952467-0.35355339j]]


In [11]:
for i in range(len(values)):
    if np.round(values[i],5) == 1 + 0j:
        pi_1 = vectors.T[i]
        break
pi_1

array([-0.58746336+0.j, -0.35247801+0.j, -0.72845456+0.j])

#### Normalising `pi_1`

In [12]:
pi_1 = pi_1/np.sum(pi_1)
pi_1

array([0.35211268-0.j, 0.21126761-0.j, 0.43661972-0.j])

In [13]:
print(f'Stationary Distribution = {pi_1.real}')

Stationary Distribution = [0.35211268 0.21126761 0.43661972]


# Probability of a Sequence
$$P(\text{Pizza} --> \text{Hot Dog} --> \text{Hot Dog} --> \text{Burger})=\text{ }?$$

In [14]:
def find_prob(seq, A, pi):
    start_state = seq[0]
    prob = pi[start_state]
    prev_state = start_state
    for i in range(1, len(seq)):
        curr_state = seq[i]
        prob *= A[prev_state][curr_state]
        prev_state = curr_state
    return prob

In [15]:
print(f'P(Pizza --> Hot Dog --> Hot Dog --> Burger) = {find_prob([1,2,2,0], A, pi_1.real):.5f}')

P(Pizza --> Hot Dog --> Hot Dog --> Burger) = 0.03697
